# Veeper: Create, Run, and Collate

This is a generalized code that can bring in any spectra, save it to its own components folder, and save the corresponding fits

Install: https://github.com/mattcwilde/veeper

This notebook is supposed to be run after asborption features have been identified using pyigm_igm guesses

Install: https://github.com/pyigm/pyigm

Here are some notes for the future

#### Running this code in Step 3 will produce a FitInspection.pdf file with a preview of your voigt fits: 
- Red line = total 
- dashed line = particular component 
- the order is the redshift 

#### If the file needs to be changed (editing.ipynb): 

In [1]:
##--- Step one 
from joebvp.utils import pyigm_to_veeper
from joebvp.VPmeasure import batch_fit
from joebvp import VPmeasure
import glob
import os
import astropy.units as u

read_sets: Using set file -- 
  /Users/samgarza/linetools/linetools/lists/sets/llist_v1.3.ascii
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/morton03_table2.fits.gz
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/verner96_tab1.fits.gz
joebvp.makevoigt: No local joebvp_cfg.py found, using default cfg.py file from joebvp.
joebvp.VPmeasure: No local joebvp_cfg.py found, using default cfg.py file from joebvp.
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/morton03_table2.fits.gz
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/verner96_tab1.fits.gz
joebvp.makevoigt: No local joebvp_cfg.py found, usin

#### Paths for each of my spectra: 
- ##### NGC 3489 / J1101: 
    - json_path =  '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1101+1429/SamIDs_jess_Nov29_2022.json' 
    - norm = 'J1101+1429_nbin3_jwnorm.fits'

- ##### NGC 4026 / J1159: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1159+5106/SamIDs_meth3_Dec1_2022_broad.json'
    - norm = 'J1159+5106_nbin3_jwnorm_meth3.fits'
    
- ##### NGC 4258 / J1222: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1222+4612/SamIDs_Jan5_2022.json'
    - norm = 'J1222+4612_nbin3_jwnorm.fits'
    
- ##### NGC 4736 / J1249: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1249+4122/SamIDs_Jan6_2022.json'
    - norm = 'J1249+4122_nbin3_jwnorm.fits'
    
- ##### NGC 3414 / 1051: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1051+2805/SamIDs_Nov29_2022.json'
    - norm = 'J1051+2805_nbin3_jwnorm.fits'
    
- ##### NGC 3627 / J1123: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123+1257/SamIDs_Dec1_2022_mwfix.json'
    - norm = 'J1123+1257_nbin3_BASIC_jwnorm.fits'

- ##### NGC 1097 / J0246: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J0246-3007/SamIDs_Nov28_2022.json'
    - norm = 'J0246-3007_nbin3_allvisits_jwnorm.fits'

- ##### NGC 4564 / J1235: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1235+1123/SamIDs_Feb27_2023.json'
    - norm = 'J1235+1123_nbin3_allvisits_jwnorm.fits'

In [3]:
#Step 2: Imput the specifications 
    # basically takes the gueses and the norm to start the process
    #dv_expand is how much you can expand the redlines to be of some problem 

#These will change for each galaxy/line of sight  
#path to specific .json file
json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1235+1123/SamIDs_Feb27_2023.json'

# normalization file that you want to use
norm = 'J1235+1123_nbin3_allvisits_jwnorm.fits'
norm_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1235+1123/J1235+1123_nbin3_allvisits_jwnorm.fits' #path to normalization file 

pyigm_to_veeper(json_path, norm, dv_expand=50*u.km/u.s)

Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


In [4]:
input_files = glob.glob('component_groups/*.txt')

#This was 10 in the other one but I want to make this 20 since it skipped something that was 15 before 
max_len = 20 #skip over files where there are more than this many lines to fit 
redo = False

to_do = []
for input_file in input_files:
    if 'rchi2' in input_file:
        continue
    if not redo:
        #already done?
        if os.path.exists(input_file.replace('.txt', '.VP')): 
            print('Skipping: Already done {}'.format(input_file))
            continue
    
    #too long?
    with open(input_file, 'r') as f:
        length = len(f.readlines())
        if length > max_len:
            print('Skipping: Too long ({} rows) {}'.format(length, input_file))
            continue
            
    to_do.append(input_file)

In [5]:
###--- Step 3: Run the fit
VPmeasure.batch_fit(norm_path, filelist=to_do, filepath='./component_groups/')

`ftol` termination condition is satisfied.
Function evaluations 8, initial cost 7.0431e+01, final cost 5.8565e+01, first-order optimality 6.13e-03.

Fit results: 

1393.76	 -0.000160	 13.766	 47.700	 26.955
 	  	  	 0.064	 8.558	 6.04 

1402.77	 -0.000160	 13.766	 47.700	 26.955
 	  	  	 0.0	 0.0	 0.0 


Reduced chi-squared: 1.074590
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_0.txt
Line parameters written to:
component_groups/input_group_0.VP
Wrote spectrum to component_groups/input_group_0_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
joebvp.joebvpfit: No local joebvp_cfg.py found, using default cfg.py file form joebvp.


Wrote spectrum to component_groups/input_group_0_inspect.fits
`ftol` termination condition is satisfied.
Function evaluations 12, initial cost 1.5251e+01, final cost 1.4622e+01, first-order optimality 3.24e-03.

Fit results: 

1215.67	 0.219040	 13.865	 49.562	 6.1792
 	  	  	 0.092	 12.735	 8.838 


Reduced chi-squared: 0.769587
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_1.txt
Line parameters written to:
component_groups/input_group_1.VP
Wrote spectrum to component_groups/input_group_1_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_1_inspect.fits
`ftol` termination condition is satisfied.
Function evaluations 10, initial cost 2.3213e+01, final cost 1.1610e+01, first-order optimality 2.55e-04.

Fit results: 

1526.71	 -0.000010	 14.765	 31.437	 -17.092
 	  	  	 0.282	 8.277	 4.093 


Reduced chi-squared: 0.515987
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_3.txt
Line parameters written to:
component_groups/input_group_3.VP
Wrote spectrum to component_groups/input_group_3_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_3_inspect.fits
`ftol` termination condition is satisfied.
Function evaluations 10, initial cost 1.6789e+01, final cost 1.3757e+01, first-order optimality 1.57e-03.

Fit results: 

1215.67	 0.224910	 14.204	 60.800	 8.3556
 	  	  	 0.07	 9.217	 6.451 


Reduced chi-squared: 0.687867
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_2.txt
Line parameters written to:
component_groups/input_group_2.VP
Wrote spectrum to component_groups/input_group_2_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_2_inspect.fits
There are no valid pixels to fit in the region. Moving on.
`ftol` termination condition is satisfied.
Function evaluations 13, initial cost 6.5588e+01, final cost 5.4588e+01, first-order optimality 5.50e-04.

Fit results: 

1548.2	 0.004400	 14.025	 85.000	 -20.502
 	  	  	 0.209	 50.446	 35.449 

1550.78	 0.004400	 14.025	 85.000	 -20.502
 	  	  	 0.0	 0.0	 0.0 

1548.2	 0.005150	 14.234	 65.500	 -72.329
 	  	  	 0.13	 23.543	 16.739 

1550.78	 0.005150	 14.234	 65.500	 -72.329
 	  	  	 0.0	 0.0	 0.0 


Reduced chi-squared: 0.742696
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_5.txt
Line parameters written to:
component_groups/input_group_5.VP
Wrote spectrum to component_groups/input_group_5_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_5_inspect.fits
`ftol` termination condition is satisfied.
Function evaluations 11, initial cost 6.3157e+01, final cost 4.6065e+01, first-order optimality 9.96e-04.

Fit results: 

1548.2	 0.000120	 14.583	 52.700	 -24.668
 	  	  	 0.055	 6.09	 5.16 

1550.78	 0.000120	 14.583	 52.700	 -24.668
 	  	  	 0.0	 0.0	 0.0 


Reduced chi-squared: 0.780768
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_4.txt
Line parameters written to:
component_groups/input_group_4.VP
Wrote spectrum to component_groups/input_group_4_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_4_inspect.fits

VPmeasure: 6/7 fits converged, 0/7 failed (see log for details).

VPmeasure: concatenating individual outputs and creating figures for inspection.
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(),

Wrote spectrum to FitInspection.fits
VPmeasure: Done.


/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)


In [6]:
#make the directory to hold creates files for each qso/LOS 

parent_path = '/Users/samgarza/Dropbox/Mac/Documents/COS-Holes/veeper_and_voigt/' #where I want the created files to be put in general 
qso_name = 'J1235/' #need to change this to match each QSO LOS name; where I want the created files to be put in specifically 

#set directory name
directory = qso_name
# Path
path = os.path.join(parent_path, directory)
#Make it happen
os.mkdir(path)

In [7]:
#Step 4: Collate
import pandas as pd

vp_file_list = glob.glob('component_groups/*.VP')
vp_file_list

['component_groups/input_group_2.VP',
 'component_groups/input_group_3.VP',
 'component_groups/input_group_4.VP',
 'component_groups/input_group_0.VP',
 'component_groups/input_group_1.VP',
 'component_groups/input_group_5.VP']

In [8]:
vp_dfs = []

for vp_file in vp_file_list:
    vp_df = pd.read_csv(vp_file, sep='|')
    vp_dfs.append(vp_df)
    
all_vp_df = pd.concat(vp_dfs, ignore_index=True)
all_vp_df

,specfile,restwave,zsys,col,sigcol,bval,sigbval,vel,sigvel,nflag,...,vlim1,vlim2,wobs1,wobs2,pix1,pix2,z_comp,trans,rely,comment
0,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1235...,1215.6700,0.22491,14.204,0.070,60.800,9.217,8.356,6.451,2,...,-83.6542,83.6542,1488.745988,1489.424430,2882,2901,0.22494,H I,c,--
1,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1235...,1526.7070,-0.00001,14.765,0.282,31.437,8.277,-17.092,4.093,2,...,-101.0149,101.0149,1526.174608,1527.203454,3902,3930,-0.00007,SiII,a,--
2,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1235...,1548.2040,0.00012,14.583,0.055,52.700,6.090,-24.668,5.160,2,...,-144.6970,144.6970,1547.639359,1549.133863,4487,4528,0.00004,C IV,a,--
3,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1235...,1550.7810,0.00012,14.583,0.000,52.700,0.000,-24.668,0.000,2,...,-144.6970,144.6970,1550.215419,1551.712410,4558,4598,0.00004,C IV,a,--
4,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1235...,1393.7602,-0.00016,13.766,0.064,47.700,8.558,26.955,6.040,2,...,-141.0569,141.0569,1392.876465,1394.188036,269,305,-0.00007,SiIV,b,--
5,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1235...,1402.7729,-0.00016,13.766,0.000,47.700,0.000,26.955,0.000,2,...,-141.0569,141.0569,1401.883451,1403.203502,515,551,-0.00007,SiIV,b,--
6,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1235...,1215.6700,0.21904,13.865,0.092,49.562,12.735,6.179,8.838,2,...,-77.3168,77.3168,1481.635995,1482.263041,2688,2705,0.21906,H I,c,--
7,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1235...,1548.2040,0.00440,14.025,0.209,85.000,50.446,-20.502,35.449,2,...,-106.4752,106.4752,1554.464654,1555.564383,4673,4703,0.00433,C IV,c,--
8,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1235...,1550.7810,0.00440,14.025,0.000,85.000,0.000,-20.502,0.000,2,...,-106.4752,106.4752,1557.052075,1558.153635,4744,4774,0.00433,C IV,c,--
9,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1235...,1548.2040,0.00515,14.234,0.130,65.500,23.543,-72.329,16.739,3,...,-109.2053,109.2053,1555.616972,1556.744899,4705,4735,0.00491,C IV,c,--


In [9]:
#Save collate informtion to csv file 
all_vp_df.to_csv(path + 'all_vp.csv')

In [10]:
import shutil

#Move over the component_groups
shutil.move(parent_path + 'component_groups/', path)

'/Users/samgarza/Dropbox/Mac/Documents/COS-Holes/veeper_and_voigt/J1235/component_groups'

In [11]:
#list of created files that need to be moved over to QSO LOS specific directory 
files_to_move = ['_VP_log.dat', 'all_VP.txt', 'compiledVPoutputs.dat', 'FitInspection.fits', 'FitInspection.pdf']

# iterate files
for file in files_to_move:
    # construct full file path
    source = parent_path + file
    destination = path + file
    # move file
    shutil.move(source, destination)
    print('Moved:', file)

Moved: _VP_log.dat
Moved: all_VP.txt
Moved: compiledVPoutputs.dat
Moved: FitInspection.fits
Moved: FitInspection.pdf
